Potential data sources:

* [2010 Census Blocks with Geographic Codes Southwestern PA](https://data.wprdc.org/dataset/2010-census-blocks-with-geographic-codes-southwestern-pa)--neighborhood names
* [Housing Market Value Analysis 2021](https://data.wprdc.org/dataset/market-value-analysis-2021), 2017-19
* [Housing Indicators](https://data.wprdc.org/dataset/housing-indicators), 2012-2021, tons of info here
  * Profiles Format - foreclosure 2009-2021
  * Parcels-to-sales ratio, 2012-2021
  * Condo Sales Price 2021\$ Tract, 2012-2021
  * Condo Sales Tract, 2012-2021
  * Average Sale Price Tract in 2021 \$ 1 to 2 unit, 2012-2021
  * HMDA_By_Race might be interesting to compare with MVA category
* [PA Vacant Addresses](https://data.wprdc.org/dataset/vacant-addresses), 2012-2021

In [1]:
import numpy as np
import pandas as pd

# Data preprocessing: Census

Target fields: 
* Population age ranges
* Education (high school and bachelor)
* Moved in last decade
* Unemployed
* Income household mean
* 3 race types (White, Black, Asian) + Latino ethnicity
* Household average size


## Population at block-group (bg) level, 2010

For weighting of tract-level classes.

In [2]:
pop_2010 = pd.read_csv("data/census_raw/DECENNIALSF12010.P1-Data.csv")
pop_2010 = pop_2010.iloc[2:, [0, 2]]
pop_2010["P001001"] = pd.to_numeric(pop_2010["P001001"], errors="coerce")
pop_2010["geoid"] = pop_2010["GEO_ID"].str.replace('.*US', '', regex=True)
pop_2010 = pop_2010.iloc[:, [2, 1]].rename(columns={"geoid": "geoid", "P001001": "pop_geoid_2010"})
pop_2010

,geoid,pop_geoid_2010
2,420030103001,2594
3,420030103002,3336
4,420030103003,353
5,420030103004,317
6,420030201001,1435
...,...,...
1097,420039810001,4
1098,420039811001,7
1099,420039812001,0
1100,420039818001,461


In [3]:
pop_2010.to_csv("data/census_clean/pop_tract_2010_P1.csv", index=False)

## Age

### 2012

In [4]:
sex_age = pd.read_csv("data/census_raw/ACSST5Y2012.S0101-Data.csv")
sex_age["tract"] = sex_age["GEO_ID"].str.replace('.*US', '', regex=True)
sex_age = sex_age.iloc[:, [-1, 2, 8, 14, 20, 26, 32, 38, 44, 50, 56, 62, 68, 74, 80, 86, 92, 98, 104, 110]]
sex_age = sex_age.rename(columns = {"S0101_C01_001E": "pop_total",
                          "S0101_C01_002E": "pop_age_under5",
                          "S0101_C01_003E": "pop_age_5to9",
                          "S0101_C01_004E": "pop_age_10to14",
                          "S0101_C01_005E": "pop_age_15to19",
                          "S0101_C01_006E": "pop_age_20to24",
                          "S0101_C01_007E": "pop_age_25to29",
                          "S0101_C01_008E": "pop_age_30to34",
                          "S0101_C01_009E": "pop_age_35to39",
                          "S0101_C01_010E": "pop_age_40to44",
                          "S0101_C01_011E": "pop_age_45to49",
                          "S0101_C01_012E": "pop_age_50to54",
                          "S0101_C01_013E": "pop_age_55to59",
                          "S0101_C01_014E": "pop_age_60to64",
                          "S0101_C01_015E": "pop_age_65to69",
                          "S0101_C01_016E": "pop_age_70to74",
                          "S0101_C01_017E": "pop_age_75to79",
                          "S0101_C01_018E": "pop_age_80to84",
                          "S0101_C01_019E": "pop_age_85plus"
                         })
sex_age.head()

,tract,pop_total,pop_age_under5,pop_age_5to9,pop_age_10to14,pop_age_15to19,pop_age_20to24,pop_age_25to29,pop_age_30to34,pop_age_35to39,pop_age_40to44,pop_age_45to49,pop_age_50to54,pop_age_55to59,pop_age_60to64,pop_age_65to69,pop_age_70to74,pop_age_75to79,pop_age_80to84,pop_age_85plus
0,Geography,Total!!Estimate!!Total population,Total!!Estimate!!AGE!!Under 5 years,Total!!Estimate!!AGE!!5 to 9 years,Total!!Estimate!!AGE!!10 to 14 years,Total!!Estimate!!AGE!!15 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 39 years,Total!!Estimate!!AGE!!40 to 44 years,Total!!Estimate!!AGE!!45 to 49 years,Total!!Estimate!!AGE!!50 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 69 years,Total!!Estimate!!AGE!!70 to 74 years,Total!!Estimate!!AGE!!75 to 79 years,Total!!Estimate!!AGE!!80 to 84 years,Total!!Estimate!!AGE!!85 years and over
1,42003,1224772,5.2,5.4,5.5,6.5,7.1,7.1,6.0,5.8,6.2,7.2,8.0,7.3,6.0,4.4,3.5,3.1,2.9,2.9
2,42003010300,5338,0.0,1.0,0.1,38.3,35.6,3.4,3.8,4.0,1.5,2.7,2.8,3.3,1.2,0.8,0.6,0.1,0.7,0.2
3,42003020100,4385,0.2,0.0,0.5,15.1,18.9,13.6,9.1,7.0,4.0,5.8,6.2,4.3,4.3,2.6,1.6,2.2,2.7,2.1
4,42003020300,604,0.7,4.0,0.0,0.0,7.0,23.5,24.0,4.6,0.7,12.9,7.6,7.0,4.0,0.8,0.2,0.0,3.1,0.0


In [5]:
sex_age.to_csv("data/census_clean/age_2012_S0101.csv", index=False)

## 2020

In [6]:
sex_age_2020 = pd.read_csv("data/census_raw/ACSST5Y2020.S0101-Data.csv")
sex_age_2020["tract"] = sex_age_2020["GEO_ID"].str.replace('.*US', '', regex=True)
sex_age_2020 = sex_age_2020.iloc[1:, [-1, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 
                                     104, 106, 108, 110, 112, 114]]
sex_age_2020 = sex_age_2020.rename(columns = {"S0101_C01_001E": "pop_total",
                          "S0101_C02_002E": "pop_age_under5",
                          "S0101_C02_003E": "pop_age_5to9",
                          "S0101_C02_004E": "pop_age_10to14",
                          "S0101_C02_005E": "pop_age_15to19",
                          "S0101_C02_006E": "pop_age_20to24",
                          "S0101_C02_007E": "pop_age_25to29",
                          "S0101_C02_008E": "pop_age_30to34",
                          "S0101_C02_009E": "pop_age_35to39",
                          "S0101_C02_010E": "pop_age_40to44",
                          "S0101_C02_011E": "pop_age_45to49",
                          "S0101_C02_012E": "pop_age_50to54",
                          "S0101_C02_013E": "pop_age_55to59",
                          "S0101_C02_014E": "pop_age_60to64",
                          "S0101_C02_015E": "pop_age_65to69",
                          "S0101_C02_016E": "pop_age_70to74",
                          "S0101_C02_017E": "pop_age_75to79",
                          "S0101_C02_018E": "pop_age_80to84",
                          "S0101_C02_019E": "pop_age_85plus"
                         })
sex_age_2020.head()

,tract,pop_age_under5,pop_age_5to9,pop_age_10to14,pop_age_15to19,pop_age_20to24,pop_age_25to29,pop_age_30to34,pop_age_35to39,pop_age_40to44,pop_age_45to49,pop_age_50to54,pop_age_55to59,pop_age_60to64,pop_age_65to69,pop_age_70to74,pop_age_75to79,pop_age_80to84,pop_age_85plus
1,42003010301,0.0,0.0,0.0,3.4,15.9,24.0,17.4,11.2,9.1,8.0,3.8,4.1,1.4,1.0,0.7,0.0,0.0,0.0
2,42003010302,1.2,0.8,0.0,47.9,36.1,3.3,2.3,1.1,1.2,0.9,1.1,1.9,0.7,0.3,0.1,0.5,0.3,0.3
3,42003020100,1.1,1.2,0.6,15.3,16.0,14.6,13.4,3.1,4.3,2.8,2.1,6.9,6.9,4.8,3.2,2.6,0.6,0.5
4,42003020300,2.3,0.4,0.0,1.0,2.8,29.4,21.2,18.9,2.0,2.0,8.7,5.5,1.2,2.3,0.3,0.7,1.0,0.4
5,42003030500,2.1,3.8,2.1,5.6,15.0,18.0,9.7,4.0,5.6,4.3,3.5,4.4,4.0,4.5,5.6,2.0,2.3,3.3


In [7]:
sex_age_2020.to_csv("data/census_clean/age_2020_S0101.csv", index=False)

## Educational attainment

### 2012

In [8]:
ed_attainment = pd.read_csv("data/census_raw/ACSST5Y2012.S1501-Data.csv")
ed_attainment["tract"] = ed_attainment["GEO_ID"].str.replace('.*US', '', regex=True)
ed_attainment = ed_attainment.iloc[2:, [-1, 80, 86]] # % HS or higher, % bachelor's or higher
ed_attainment = ed_attainment.rename(columns = {"S1501_C01_014E": "edu_hsplus",
                                "S1501_C01_015E": "edu_bachelorsplus"})

ed_attainment.head()

,tract,edu_hsplus,edu_bachelorsplus
2,42003010300,81.5,11.9
3,42003020100,93.9,37.7
4,42003020300,93.4,75.3
5,42003030500,76.1,13.2
6,42003040200,89.5,38.3


In [9]:
ed_attainment.to_csv("data/census_clean/edu_attainment_2012_S1501.csv", index=False)

## 2020

In [10]:
ed_attainment_2020 = pd.read_csv("data/census_raw/ACSST5Y2020.S1501-Data.csv")
ed_attainment_2020["tract"] = ed_attainment_2020["GEO_ID"].str.replace('.*US', '', regex=True)
ed_attainment_2020 = ed_attainment_2020.iloc[1:, [-1, 146, 158]] # % HS or higher, % bachelor's or higher
ed_attainment_2020 = ed_attainment_2020.rename(columns = {"S1501_C02_009E": "edu_hsplus",
                                "S1501_C02_015E": "edu_bachelorsplus"})

ed_attainment_2020.head()

,tract,edu_hsplus,edu_bachelorsplus
1,42003010301,45.3,2.4
2,42003010302,21.0,34.5
3,42003020100,4.1,73.3
4,42003020300,4.0,88.4
5,42003030500,32.0,38.6


In [11]:
ed_attainment_2020.to_csv("data/census_clean/edu_attainment_2020_S1501.csv", index=False)

## Housing units

### 2012

In [12]:
housing_units = pd.read_csv("data/census_raw/ACSDT5Y2012.B25001-2023-11-30T021533.csv")

housing_units["Total"] = housing_units["Total"].fillna(method="bfill")
housing_units = housing_units[housing_units["Label (Grouping)"] != "    Estimate"]

# isolate tract number
housing_units["tract_num"] = housing_units["Label (Grouping)"].str.replace('Census Tract ', '', regex=True)
housing_units["tract_num"] = housing_units["tract_num"].str.replace(', Allegheny County, Pennsylvania', '', regex=True)
housing_units = housing_units.rename(columns={"Total": "housing_units_total"})

# geoid
allegheny = 42003
geoids = housing_units["tract_num"].str.replace('.','')
geoids = geoids.apply(lambda x: "42003" + ("0" + str(x) if len(str(x)) == 3 else str(x)) + "00")
geoids = geoids.str[:11]
housing_units["tract"] = geoids
housing_units = housing_units.iloc[:, [-1, 1]]

housing_units.head()

C:\Users\jessk\AppData\Local\Temp\ipykernel_21908\3634717536.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  geoids = housing_units["tract_num"].str.replace('.','')


,tract,housing_units_total
0,42003010300,397
2,42003020100,"2,076"
4,42003020300,430
6,42003030500,"1,453"
8,42003040200,781


In [13]:
housing_units.to_csv("data/census_clean/housing_units_2012_B25001.csv", index=False)

### 2020

In [14]:
housing_units_2020 = pd.read_csv("data/census_raw/ACSDT5Y2020.B25001-Data.csv")

housing_units_2020["tract"] = housing_units_2020["GEO_ID"].str.replace('.*US', '', regex=True)
housing_units_2020 = housing_units_2020.iloc[1:, [-1, 2]] 

housing_units_2020 = housing_units_2020.rename(columns = {"B25001_001E": "housing_units_total"})

housing_units_2020.head()

,tract,housing_units_total
1,42003010301,0
2,42003010302,493
3,42003020100,2643
4,42003020300,1108
5,42003030500,1865


In [15]:
housing_units_2020.to_csv("data/census_clean/housing_units_2020_B25001.csv", index=False)

## Selected economic characteristics

### 2012

In [16]:
econ = pd.read_csv("data/census_raw/ACSDP5Y2012.DP03-Data.csv")

econ["tract"] = econ["GEO_ID"].str.replace('.*US', '', regex=True)
# econ = econ.iloc[:, [-1, 8, 36, 60, 76, 84, 88, 96, 98, 108, 112, 116, 246, 250, 384, 476]]
econ = econ.iloc[:, [-1, 36, 250]]

# Drop rows containing "-" in any column
econ = econ[~econ.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

# econ = econ.rename(columns = {"DP03_0002PE": "pct_in_labor_force",
#                               "DP03_0009PE": "unemployed",
#                               "DP03_0015PE": "all_parents_in_labor_force",
#                               "DP03_0019PE": "commute_drove_alone",
#                               "DP03_0021PE": "commute_public",
#                               "DP03_0022PE": "commute_walked",
#                               "DP03_0024PE": "commute_wfh",
#                               "DP03_0025E": "commute_time_mean",
#                               "DP03_0027PE": "occ_mgmt_bus_sci_art",
#                               "DP03_0028PE": "occ_service",
#                               "DP03_0029PE": "occ_sales_office",
#                               "DP03_0062E": "income_household_median",
#                               "DP03_0063E": "income_household_mean",
#                               "DP03_0096PE": "health_insurance",
#                               "DP03_0119PE": "income_below_poverty"
#                          })
econ = econ.rename(columns = {"DP03_0009PE": "unemployed",
                              "DP03_0063E": "income_household_mean"
                         })

econ.sample(5)


,tract,unemployed,income_household_mean
339,42003523800,4.7,60092
235,42003473602,4.3,127250
48,42003130400,16.8,30898
11,42003050600,16.4,46979
189,42003451300,5.9,113216


In [17]:
econ.to_csv("data/census_clean/econ_selected_2012_DP03.csv", index=False)

### 2020

In [18]:
econ_2020 = pd.read_csv("data/census_raw/ACSDP5Y2020.DP03-Data.csv")

econ_2020["tract"] = econ_2020["GEO_ID"].str.replace('.*US', '', regex=True)
econ_2020 = econ_2020.iloc[:, [-1, 284, 126]]

# Drop rows containing "-" in any column
econ_2020 = econ_2020[~econ_2020.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

econ_2020 = econ_2020.rename(columns = {"DP03_0005PE": "unemployed",
                              "DP03_0063E": "income_household_mean"
                         })

econ_2020.sample(5)


,tract,unemployed,income_household_mean
49,42003140300,2.1,156437
386,42003980600,0.0,N
132,42003417200,9.0,66264
379,42003565200,4.6,57728
125,42003414101,1.3,131909


In [19]:
econ_2020.to_csv("data/census_clean/econ_selected_2020_DP03.csv", index=False)

## Race and ethnicity

### 2012

In [20]:
race_ethnicity = pd.read_csv("data/census_raw/ACSDP5Y2012.DP05-Data.csv")

race_ethnicity["tract"] = race_ethnicity["GEO_ID"].str.replace('.*US', '', regex=True)
race_ethnicity = race_ethnicity.iloc[2:, [-1, 128, 132, 156, 264]]

# Drop rows containing "-" in any column
race_ethnicity = race_ethnicity[~race_ethnicity.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

race_ethnicity = race_ethnicity.rename(
    columns = {"DP05_0032PE": "race_white_mono",
               "DP05_0033PE": "race_black_mono",
               "DP05_0039PE": "race_asian_mono",
               "DP05_0066PE": "ethnicity_latino"})

race_ethnicity.shape

(396, 5)

In [21]:
race_ethnicity.to_csv("data/census_clean/race_ethnicity_2012_DP05.csv", index=False)

### 2020

In [22]:
race_ethnicity_2020 = pd.read_csv("data/census_raw/ACSDP5Y2020.DP05-Data.csv")

race_ethnicity_2020["tract"] = race_ethnicity_2020["GEO_ID"].str.replace('.*US', '', regex=True)
race_ethnicity_2020 = race_ethnicity_2020.iloc[1:, [-1, 252, 254, 266, 320]]

# Drop rows containing "-" in any column
race_ethnicity_2020 = race_ethnicity_2020[~race_ethnicity_2020.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

race_ethnicity_2020 = race_ethnicity_2020.rename(
    columns = {"DP05_0037PE": "race_white_mono",
               "DP05_0038PE": "race_black_mono",
               "DP05_0044PE": "race_asian_mono",
               "DP05_0071PE": "ethnicity_latino"})

race_ethnicity_2020.head()

,tract,race_white_mono,race_black_mono,race_asian_mono,ethnicity_latino
1,42003010301,36.4,56.2,0.0,3.5
2,42003010302,76.5,12.2,6.1,4.0
3,42003020100,73.6,11.0,11.9,4.3
4,42003020300,84.1,1.8,10.8,4.1
5,42003030500,27.9,60.1,8.5,4.4


In [23]:
race_ethnicity.to_csv("data/census_clean/race_ethnicity_2020_DP05.csv", index=False)

## Marital status

### 2012

In [24]:
marital = pd.read_csv("data/census_raw/ACSST5Y2012.S1201-Data.csv")
    
marital["tract"] = marital["GEO_ID"].str.replace('.*US', '', regex=True)
marital = marital.iloc[:, [-1, 4, 8, 12]]

# Drop rows containing "-" in any column
marital = marital[~marital.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

marital = marital.rename(
    columns = {"S1201_C02_001E": "mar_married",
               "S1201_C04_001E": "mar_divorced",
               "S1201_C06_001E": "mar_never"})

marital.shape

(398, 4)

In [25]:
marital.to_csv("data/census_clean/marital_2012_S1201.csv", index=False)

## Selected social characteristics

### 2012

In [26]:
social = pd.read_csv("data/census_raw/ACSDP5Y2012.DP02-Data.csv")
    
social["tract"] = social["GEO_ID"].str.replace('.*US', '', regex=True)
# social = social.iloc[:, [-1, 40, 52, 56, 58, 316]]
social = social.iloc[1:, [-1, 58]]

# Drop rows containing "-" in any column
social = social[~social.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

# social = social.rename(
#     columns = {"DP02_0010PE": "households_nonfamily",
#                "DP02_0013PE": "households_under18",
#                "DP02_0014PE": "households_over65",
#                "DP02_0015E": "households_avg_size",
#                "DP02_0079PE": "residence_same_1yrago"})
social = social.rename(
    columns = {"DP02_0015E": "households_avg_size"})

social.head()

,tract,households_avg_size
1,42003010300,1.80
2,42003020100,1.33
3,42003020300,1.59
4,42003030500,1.75
5,42003040200,2.18


In [27]:
social.to_csv("data/census_clean/social_2012_DP02.csv", index=False)

### 2020

In [28]:
social_2020 = pd.read_csv("data/census_raw/ACSDP5Y2020.DP02-Data.csv")
    
social_2020["tract"] = social_2020["GEO_ID"].str.replace('.*US', '', regex=True)
social_2020 = social_2020.iloc[1:, [-1, 32]]

# Drop rows containing "-" in any column
social_2020 = social_2020[~social_2020.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

social_2020 = social_2020.rename(
    columns = {"DP02_0016E": "households_avg_size"})

social_2020.head()

,tract,households_avg_size
2,42003010302,1.96
3,42003020100,1.49
4,42003020300,1.38
5,42003030500,1.69
6,42003040200,2.03


In [29]:
social_2020.to_csv("data/census_clean/social_2020_DP02.csv", index=False)

## Mortgages

### 2012

In [30]:
mortgages = pd.read_csv("data/census_raw/ACSST5Y2012.S2506-Data.csv")
    
mortgages["tract"] = mortgages["GEO_ID"].str.replace('.*US', '', regex=True)
mortgages = mortgages.iloc[1:, [-1, 18, 28, 136]]

# Drop rows containing "-" in any column
mortgages = mortgages[~mortgages.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

mortgages = mortgages.rename(
    columns = {"S2506_C01_009E": "mortgaged_value_median",
               "S2506_C01_014E": "mortgaged_no2nd_noheq",
               "S2506_C01_068E": "real_estate_taxes_median"})

mortgages.shape

(386, 4)

In [31]:
mortgages.to_csv("data/census_clean/mortgages_2012_S2506.csv", index=False)

## Selected housing characteristics

### 2012

In [32]:
housing = pd.read_csv("data/census_raw/ACSDP5Y2012.DP04-Data.csv")
    
housing["tract"] = housing["GEO_ID"].str.replace('.*US', '', regex=True)
#housing = housing.iloc[:, [-1, 18, 184, 204]]
housing = housing.iloc[1:, [-1, 204]]

# Drop rows containing "-" in any column
housing = housing[~housing.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

# housing = housing.rename(
#     columns = {"DP04_0005E": "rental_vacancy_rate",
#                "DP04_0046PE": "renter_occupied",
#                "DP04_0051PE": "movedin_last_decade"})
housing = housing.rename(
    columns = {"DP04_0051PE": "movedin_last_decade"})

housing.head()

,tract,movedin_last_decade
1,42003010300,40.3
2,42003020100,58.2
3,42003020300,78.2
4,42003030500,59.7
5,42003040200,51.2


In [33]:
housing.to_csv("data/census_clean/housing_2012_DP04.csv", index=False)

### 2020

In [34]:
housing_2020 = pd.read_csv("data/census_raw/ACSDP5Y2020.DP04-Data.csv")
    
housing_2020["tract"] = housing_2020["GEO_ID"].str.replace('.*US', '', regex=True)
housing_2020 = housing_2020.iloc[1:, [-1, 388, 390, 392]]

# Drop rows containing "-" in any column
housing_2020 = housing_2020[~housing_2020.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

# housing_2020 = housing_2020.rename(
#     columns = {"DP04_0051PE": "movedin_last_decade"})
housing_2020["movedin_last_decade"] = housing_2020.DP04_0051PE.astype('float') + housing_2020.DP04_0052PE.astype('float') + housing_2020.DP04_0053PE.astype('float')
housing_2020 = housing_2020.iloc[:, [0,-1]]
housing_2020.head()

,tract,movedin_last_decade
2,42003010302,86.9
3,42003020100,85.4
4,42003020300,95.4
5,42003030500,61.8
6,42003040200,62.5


In [35]:
housing_2020.to_csv("data/census_clean/housing_2020_DP04.csv", index=False)

## Join 2012 data

In [36]:
# census_2012 = sex_age.merge(ed_attainment, on="tract"
#                             ).merge(housing, on="tract"
#                             ).merge(econ, on="tract"
#                             ).merge(race_ethnicity, on="tract"
#                             ).merge(marital, on="tract"
#                             ).merge(social, on="tract"
#                             ).merge(mortgages, on="tract")
census_2012 = sex_age.merge(ed_attainment, on="tract"
                            ).merge(housing, on="tract"
                            ).merge(econ, on="tract"
                            ).merge(race_ethnicity, on="tract"
                            ).merge(social, on="tract"
                            )

census_2012 = census_2012[~census_2012.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

census_2012.dtypes

tract                    object
pop_total                object
pop_age_under5           object
pop_age_5to9             object
pop_age_10to14           object
pop_age_15to19           object
pop_age_20to24           object
pop_age_25to29           object
pop_age_30to34           object
pop_age_35to39           object
pop_age_40to44           object
pop_age_45to49           object
pop_age_50to54           object
pop_age_55to59           object
pop_age_60to64           object
pop_age_65to69           object
pop_age_70to74           object
pop_age_75to79           object
pop_age_80to84           object
pop_age_85plus           object
edu_hsplus               object
edu_bachelorsplus        object
movedin_last_decade      object
unemployed               object
income_household_mean    object
race_white_mono          object
race_black_mono          object
race_asian_mono          object
ethnicity_latino         object
households_avg_size      object
dtype: object

In [37]:
census_2012.to_csv("data/census_clean/census_2012.csv", index=False)

In [38]:
census_2020 = sex_age_2020.merge(ed_attainment_2020, on="tract"
                            ).merge(housing_2020, on="tract"
                            ).merge(econ_2020, on="tract"
                            ).merge(race_ethnicity_2020, on="tract"
                            ).merge(social_2020, on="tract"
                            )

census_2020 = census_2012[~census_2012.apply(lambda x: x.str.contains('-', na=False)).any(axis=1)]

census_2020

,tract,pop_total,pop_age_under5,pop_age_5to9,pop_age_10to14,pop_age_15to19,pop_age_20to24,pop_age_25to29,pop_age_30to34,pop_age_35to39,...,edu_hsplus,edu_bachelorsplus,movedin_last_decade,unemployed,income_household_mean,race_white_mono,race_black_mono,race_asian_mono,ethnicity_latino,households_avg_size
0,42003020100,4385,0.2,0.0,0.5,15.1,18.9,13.6,9.1,7.0,...,93.9,37.7,58.2,8.0,91071,59.1,27.1,7.4,5.5,1.33
1,42003020300,604,0.7,4.0,0.0,0.0,7.0,23.5,24.0,4.6,...,93.4,75.3,78.2,1.9,152763,71.2,3.5,10.8,15.4,1.59
2,42003030500,2304,6.7,4.9,8.1,5.9,16.1,5.6,3.3,4.2,...,76.1,13.2,59.7,26.0,19575,15.8,81.9,0.0,2.0,1.75
3,42003040200,2183,7.9,0.7,2.3,26.4,21.6,10.7,4.8,5.2,...,89.5,38.3,51.2,18.4,29110,44.0,48.1,4.8,2.8,2.18
4,42003040400,2298,0.4,0.0,0.0,18.8,35.2,4.4,3.8,0.0,...,98.9,63.3,51.1,1.2,52483,54.3,2.7,40.6,2.2,1.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,42003980400,22,0.0,0.0,0.0,0.0,36.4,0.0,0.0,0.0,...,85.7,0.0,76.9,0.0,N,54.5,45.5,0.0,27.3,1.38
389,42003980700,38,0.0,0.0,0.0,26.3,13.2,10.5,0.0,0.0,...,100.0,43.5,0.0,0.0,N,100.0,0.0,0.0,0.0,2.71
390,42003981000,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,0.0,100.0,0.0,N,100.0,0.0,0.0,0.0,2.00
391,42003981800,510,0.0,0.2,0.6,18.8,25.7,1.2,1.2,0.6,...,67.4,7.2,46.2,55.5,48508,49.2,45.5,0.0,3.1,1.92


In [39]:
census_2020.to_csv("data/census_clean/census_2020.csv", index=False)